In [1]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2

In [2]:
import pandas as pd

df = pd.read_csv('ramification/train_labels.csv')

df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,mask_0.jpg,640,800,ramification,8,303,64,348
1,mask_0.jpg,640,800,ramification,9,209,62,248
2,mask_0.jpg,640,800,ramification,300,223,370,276
3,mask_0.jpg,640,800,ramification,312,193,360,234
4,mask_0.jpg,640,800,ramification,312,88,357,129


In [3]:
import os
import numpy as np
import json
from detectron2.structures import BoxMode
import itertools
import cv2

def get_ramification_dicts(csv_file, img_dir):
    df = pd.read_csv(csv_file)
    df['filename'] = df['filename'].map(lambda x: img_dir+x)

    classes = ['ramification']
    df['class'] = df['class'].map(lambda x: 'ramification')
    df['class_int'] = df['class'].map(lambda x: classes.index(x))

    dataset_dicts = []
    for filename in df['filename'].unique().tolist():
        record = {}
        print(filename)
        image = cv2.imread(filename)
        print(image)
        height, width, _ = image.shape
        
        record["file_name"] = filename
        record["height"] = height
        record["width"] = width

        objs = []
        for index, row in df[(df['filename']==filename)].iterrows():
          obj= {
              'bbox': [row['xmin'], row['ymin'], row['xmax'], row['ymax']],
              'bbox_mode': BoxMode.XYXY_ABS,
              'category_id': row['class_int'],
              "iscrowd": 0
          }
          objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [4]:
from detectron2.data import DatasetCatalog, MetadataCatalog

classes = ['ramification']

for d in ["train", "test"]:
  DatasetCatalog.register('ramification_data/' + d, lambda d=d: get_ramification_dicts('ramification/' + d + '_labels.csv', 'ramification/' + d+'/'))
  MetadataCatalog.get('ramification_data/' + d).set(thing_classes=classes)
ramification_metadata = MetadataCatalog.get('ramification_data/train')


In [5]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.logger import setup_logger
setup_logger()
cfg = get_cfg()
cfg.merge_from_file("./detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = "model_final_59f53c.pkl"
cfg.DATASETS.TRAIN = ('ramification_data/train',)
cfg.DATASETS.TEST = ()   
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 4
cfg.SOLVER.MAX_ITER = 20000
cfg.MODEL.RETINANET.NUM_CLASSES = 1
cfg.SOLVER.BASE_LR = 0.0001
cfg.MODEL.DEVICE = "cuda:2"
cfg.OUTPUT_DIR = 'output_model/'
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

Loading config ./detectron2/model_zoo/configs/COCO-Detection/../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


WARNING [02/02 17:48:56 d2.config.compat]: Config './detectron2/model_zoo/configs/COCO-Detection/retinanet_R_101_FPN_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.
[02/02 17:48:58 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (co

ramification/train/mask_0.jpg
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [253 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [253 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [253 255 255]
  [255 255 255]
  [255 255 255]]]
ramification/train/mask_1.jpg
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
ramification/train/mask_147.jpg
[[[255 254 255]
  [255 254 255]
  [255 252 253]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 253 254]
  [255 254 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[251 255 255]
  [248 253 252]
  [253 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [2

[[[255 255 255]
  [255 254 254]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [252 252 252]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[250 252 252]
  [253 255 255]
  [253 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
ramification/train/mask_5.jpg
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255

...

[02/02 21:41:15 d2.utils.events]: eta: 0:39:35  iter: 17119  total_loss: 0.006  loss_cls: 0.002  loss_box_reg: 0.005  time: 0.8130  data_time: 0.0043  lr: 0.000100  max_mem: 0M
[02/02 21:41:31 d2.utils.events]: eta: 0:39:21  iter: 17139  total_loss: 0.006  loss_cls: 0.002  loss_box_reg: 0.005  time: 0.8130  data_time: 0.0044  lr: 0.000100  max_mem: 0M
[02/02 21:41:47 d2.utils.events]: eta: 0:39:08  iter: 17159  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.005  time: 0.8130  data_time: 0.0044  lr: 0.000100  max_mem: 0M
[02/02 21:42:04 d2.utils.events]: eta: 0:39:01  iter: 17179  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.005  time: 0.8130  data_time: 0.0045  lr: 0.000100  max_mem: 0M
[02/02 21:42:21 d2.utils.events]: eta: 0:38:56  iter: 17199  total_loss: 0.007  loss_cls: 0.002  loss_box_reg: 0.005  time: 0.8130  data_time: 0.0046  lr: 0.000100  max_mem: 0M
[02/02 21:42:37 d2.utils.events]: eta: 0:38:34  iter: 17219  total_loss: 0.007  loss_cls: 0.002  loss_box_reg: 0.00

[02/02 21:53:30 d2.utils.events]: eta: 0:27:36  iter: 18019  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.005  time: 0.8131  data_time: 0.0043  lr: 0.000100  max_mem: 0M
[02/02 21:53:46 d2.utils.events]: eta: 0:27:18  iter: 18039  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8131  data_time: 0.0044  lr: 0.000100  max_mem: 0M
[02/02 21:54:02 d2.utils.events]: eta: 0:26:59  iter: 18059  total_loss: 0.007  loss_cls: 0.002  loss_box_reg: 0.005  time: 0.8131  data_time: 0.0045  lr: 0.000100  max_mem: 0M
[02/02 21:54:17 d2.utils.events]: eta: 0:26:38  iter: 18079  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.005  time: 0.8130  data_time: 0.0044  lr: 0.000100  max_mem: 0M
[02/02 21:54:34 d2.utils.events]: eta: 0:26:17  iter: 18099  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.005  time: 0.8130  data_time: 0.0041  lr: 0.000100  max_mem: 0M
[02/02 21:54:50 d2.utils.events]: eta: 0:26:04  iter: 18119  total_loss: 0.006  loss_cls: 0.002  loss_box_reg: 0.00

[02/02 22:05:37 d2.utils.events]: eta: 0:14:36  iter: 18919  total_loss: 0.005  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8128  data_time: 0.0041  lr: 0.000100  max_mem: 0M
[02/02 22:05:53 d2.utils.events]: eta: 0:14:16  iter: 18939  total_loss: 0.005  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8128  data_time: 0.0043  lr: 0.000100  max_mem: 0M
[02/02 22:06:09 d2.utils.events]: eta: 0:13:54  iter: 18959  total_loss: 0.005  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8128  data_time: 0.0045  lr: 0.000100  max_mem: 0M
[02/02 22:06:26 d2.utils.events]: eta: 0:13:37  iter: 18979  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8128  data_time: 0.0044  lr: 0.000100  max_mem: 0M
[02/02 22:06:42 d2.utils.events]: eta: 0:13:22  iter: 18999  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8128  data_time: 0.0046  lr: 0.000100  max_mem: 0M
[02/02 22:06:59 d2.utils.events]: eta: 0:13:07  iter: 19019  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.00

[02/02 22:17:44 d2.utils.events]: eta: 0:02:24  iter: 19819  total_loss: 0.005  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8126  data_time: 0.0043  lr: 0.000100  max_mem: 0M
[02/02 22:18:01 d2.utils.events]: eta: 0:02:08  iter: 19839  total_loss: 0.005  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8125  data_time: 0.0044  lr: 0.000100  max_mem: 0M
[02/02 22:18:16 d2.utils.events]: eta: 0:01:52  iter: 19859  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8125  data_time: 0.0046  lr: 0.000100  max_mem: 0M
[02/02 22:18:33 d2.utils.events]: eta: 0:01:36  iter: 19879  total_loss: 0.005  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8125  data_time: 0.0045  lr: 0.000100  max_mem: 0M
[02/02 22:18:49 d2.utils.events]: eta: 0:01:20  iter: 19899  total_loss: 0.005  loss_cls: 0.001  loss_box_reg: 0.004  time: 0.8125  data_time: 0.0045  lr: 0.000100  max_mem: 0M
[02/02 22:19:05 d2.utils.events]: eta: 0:01:04  iter: 19919  total_loss: 0.006  loss_cls: 0.001  loss_box_reg: 0.00

OrderedDict()